# Bab 19: Training and Deploying TensorFlow Models (Melatih dan Menyebarkan Model TensorFlow)

### 1. Pendahuluan

Bab 19 ini adalah bab yang sangat praktis, berfokus pada transisi dari melatih model *Machine Learning* di lingkungan penelitian (misalnya, Jupyter Notebook) ke menyebarkannya di lingkungan produksi. Ini mencakup berbagai alat dan praktik terbaik untuk pelatihan skala besar, deployment, dan pemantauan. Meskipun buku ini berfokus pada TensorFlow 2, banyak konsep yang berlaku secara umum untuk alur kerja MLOps (Machine Learning Operations).

Bab ini akan membahas:
* **Distribusi Pelatihan:** Melatih model di banyak perangkat (GPU/TPU) atau di banyak mesin.
* **TensorFlow Serving:** Menyebarkan model terlatih untuk inferensi.
* **TensorFlow Lite:** Mengoptimalkan model untuk perangkat seluler dan *edge*.
* **TensorFlow.js:** Menjalankan model di browser.

### 2. Distribusi Pelatihan (Training at Scale)

Melatih model *Deep Learning* yang besar atau pada dataset yang sangat besar membutuhkan strategi distribusi pelatihan.

#### a. Paralelisme Data vs Paralelisme Model (Data Parallelism vs. Model Parallelism)
* **Paralelisme Data:** Setiap perangkat (GPU/CPU/TPU) menerima *mini-batch* data yang berbeda, menghitung gradien secara independen, dan gradien-gradien ini kemudian digabungkan (misalnya, dirata-ratakan) untuk memperbarui bobot model. Model yang sama disalin ke setiap perangkat. Ini adalah strategi yang paling umum dan mudah diimplementasikan.
* **Paralelisme Model:** Model dibagi menjadi beberapa bagian, dan setiap bagian ditempatkan pada perangkat yang berbeda. Data kemudian mengalir melalui perangkat secara berurutan. Ini lebih kompleks dan diperlukan ketika model terlalu besar untuk muat di satu perangkat.

#### b. Strategi Distribusi TensorFlow (TensorFlow Distribution Strategies)
TensorFlow 2.x menyediakan API yang mudah digunakan untuk distribusi pelatihan melalui `tf.distribute.Strategy`. Ini memungkinkan Anda menulis kode pelatihan standar, dan TensorFlow akan menanganinya secara paralel.

* **`MirroredStrategy`:** Strategi paling umum untuk *single-host, multi-GPU training*. Ini membuat salinan model pada setiap GPU dan menggunakan *all-reduce* untuk mengumpulkan gradien.
* **`MultiWorkerMirroredStrategy`:** Untuk *multi-host, multi-GPU training* (melatih di banyak mesin). Mirip dengan `MirroredStrategy` tetapi meluas ke beberapa server. Membutuhkan konfigurasi klaster.
* **`TPUStrategy`:** Untuk melatih model pada *Tensor Processing Units* (TPUs) Google. TPU adalah akselerator khusus yang dioptimalkan untuk beban kerja *deep learning*.
* **`CentralStorageStrategy`:** Bobot dan variabel disimpan di CPU pusat, dan operasi didistribusikan ke perangkat lain. Kurang efisien untuk GPU modern.
* **`ParameterServerStrategy`:** Untuk skala yang sangat besar, di mana variabel model disimpan di *parameter servers* dan pekerja mengambil/memperbarui gradien.

**Bagaimana Menggunakannya:** Anda membungkus kode pembangunan dan kompilasi model Anda dalam *scope* strategi distribusi.
`with strategy.scope(): ...`

#### c. Input Pipelines Terdistribusi (Distributed Input Pipelines)
Sangat penting untuk memastikan *pipeline* input data juga efisien dalam lingkungan terdistribusi. `tf.data` API dapat terintegrasi dengan strategi distribusi untuk memastikan data di-*shard* dengan benar dan dibaca secara paralel.
* `dataset.distribute_splits_from_function()`: Metode untuk membagi dataset di antara pekerja.

### 3. TensorFlow Serving

Setelah model dilatih, Anda perlu menyebarkannya agar dapat digunakan untuk inferensi dalam produksi. TensorFlow Serving adalah sistem yang fleksibel dan berkinerja tinggi yang dirancang untuk menyebarkan model *Machine Learning* dalam produksi.

#### a. Mengekspor Model (Exporting Models)
Model Keras dapat diekspor ke format **SavedModel** (`model.save("my_model_path")`), yang merupakan format kanonis TensorFlow untuk menyimpan model yang siap produksi. SavedModel berisi grafik komputasi TensorFlow dan bobot model.

#### b. Menjalankan TensorFlow Serving (Running TensorFlow Serving)
TensorFlow Serving adalah server yang dapat menerima permintaan inferensi melalui REST API atau gRPC.
* Anda menjalankan server TensorFlow Serving (biasanya dalam container Docker).
* Server ini "melayani" model-model yang disimpan dalam format SavedModel dari direktori yang ditentukan.
* Dapat melayani beberapa versi model secara bersamaan (untuk *A/B testing* atau *canary deployments*).

#### c. Klien TensorFlow Serving (TensorFlow Serving Client)
Aplikasi klien (misalnya, aplikasi Python, Node.js, Java) dapat mengirim permintaan inferensi ke server TensorFlow Serving.
* Permintaan gRPC (lebih cepat) atau REST (lebih mudah digunakan) dapat dikirim dengan data input.
* Server akan mengembalikan prediksi model.

### 4. TensorFlow Lite

TensorFlow Lite adalah kerangka kerja ringan yang dirancang untuk menjalankan model TensorFlow pada perangkat seluler, *embedded devices*, dan perangkat IoT (Internet of Things). Ini memungkinkan inferensi *on-device* dengan latensi rendah.

#### a. Konverter TensorFlow Lite (TensorFlow Lite Converter)
* Anda mengkonversi model TensorFlow yang sudah dilatih (dalam format SavedModel atau Keras H5) ke format TensorFlow Lite (`.tflite`).
* Konverter ini melakukan optimisasi:
    * **Quantization:** Mengurangi presisi numerik bobot dan/atau aktivasi (misalnya, dari float32 ke int8) untuk mengurangi ukuran model dan mempercepat komputasi.
    * **Pruning:** Menghapus bobot yang tidak signifikan.
    * **Graph Freezing:** Menggabungkan variabel menjadi konstanta.
* Pustaka `tf.lite.TFLiteConverter` digunakan.

#### b. Interpreter TensorFlow Lite (TensorFlow Lite Interpreter)
* Setelah dikonversi, model `.tflite` dapat dimuat dan dijalankan menggunakan `tf.lite.Interpreter` di perangkat target.
* Mendukung berbagai bahasa pemrograman (Java, Swift/Objective-C, C++, Python).

### 5. TensorFlow.js

TensorFlow.js adalah pustaka JavaScript untuk melatih dan menyebarkan model *Machine Learning* di browser web atau di Node.js.

#### a. Konversi Model (Model Conversion)
* Model Keras atau TensorFlow yang sudah dilatih dapat dikonversi ke format TensorFlow.js.
* Pustaka `tensorflowjs_converter` digunakan (instal via `pip`).

#### b. Menjalankan Model di Browser (Running Models in the Browser)
* Model yang dikonversi dapat dimuat di JavaScript menggunakan `tf.loadLayersModel()` atau `tf.loadGraphModel()`.
* Inferensi kemudian dapat dilakukan di sisi klien, memanfaatkan GPU browser (melalui WebGL).
* **Manfaat:** Privasi (data tetap di perangkat pengguna), latensi rendah (tidak perlu bolak-balik ke server), interaktivitas.
* **Kekurangan:** Batasan ukuran model dan komputasi oleh kemampuan perangkat pengguna dan browser.

### 6. TensorFlow Extended (TFX) (Gambaran Singkat)

TFX adalah *platform* end-to-end untuk membangun dan mengelola *pipeline* ML produksi. Ini menyediakan komponen untuk validasi data, *feature engineering*, pelatihan, evaluasi, dan penyebaran model. Ini melampaui cakupan detail buku ini, tetapi penting untuk MLOps yang matang.

### 7. Kesimpulan

Bab 19 menyediakan panduan penting untuk membawa model *Deep Learning* dari eksperimen ke produksi. Ini membahas strategi distribusi pelatihan TensorFlow untuk skala besar, memperkenalkan TensorFlow Serving sebagai solusi *deployment* yang kuat, dan menjelaskan TensorFlow Lite serta TensorFlow.js untuk inferensi *on-device* dan di browser. Pemahaman tentang alat-alat ini sangat penting bagi setiap insinyur ML yang ingin membangun dan mengelola sistem *deep learning* dunia nyata.

## 1. Setup

In [102]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil # For removing directories

### Loading Fashion MNIST (as used in previous chapters)


In [103]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

## 2. Training at Scale

### Data Parallelism with MirroredStrategy (Single-host, multi-GPU)

In [104]:
# Check for available GPUs
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Create a MirroredStrategy
strategy = tf.distribute.MirroredStrategy()

# Build and compile the model within the strategy's scope
with strategy.scope():
    model_mirrored = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    model_mirrored.compile(loss="sparse_categorical_crossentropy",
                           optimizer="adam",
                           metrics=["accuracy"])

# Train the model (data will be automatically sharded across devices)
model_mirrored.fit(X_train, y_train, epochs=10,
                   validation_data=(X_valid, y_valid))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.7809 - loss: 0.6090 - val_accuracy: 0.8602 - val_loss: 0.3751
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.8599 - loss: 0.3764 - val_accuracy: 0.8668 - val_loss: 0.3626
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.8736 - loss: 0.3344 - val_accuracy: 0.8834 - val_loss: 0.3192
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.8873 - loss: 0.3006 - val_accuracy: 0.8840 - val_loss: 0.3169
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8967 - loss: 0.2803 - val_accuracy: 0.8890 - val_loss: 0.3184
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9001 - loss: 0.2655 - val_accuracy: 0.8828 - val_loss: 0.3175
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9059 - loss: 0.2495 - val_accuracy: 0.8906 - val_loss: 0.3044
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9062 - loss: 0

### MultiWorkerMirroredStrategy (Multi-host, multi-GPU)

In [106]:
# This requires environment variable TF_CONFIG to be set up
# Example TF_CONFIG for worker 0 (assuming two workers):
# os.environ['TF_CONFIG'] = """
# {
#     "cluster": {
#         "worker": ["localhost:12345", "localhost:12346"]
#     },
#     "task": {"type": "worker", "index": 0}
# }
# """

# This strategy also needs a distributed dataset
def create_dataset(x_data, y_data, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).shuffle(len(x_data)).repeat()
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

multi_worker_strategy = tf.distribute.MultiWorkerMirroredStrategy()

with multi_worker_strategy.scope():
    model_multi_worker = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    model_multi_worker.compile(loss="sparse_categorical_crossentropy",
                               optimizer="adam",
                               metrics=["accuracy"])

# You would typically use `tf.data.Dataset` for input and `steps_per_epoch` for fit
train_dataset_mw = create_dataset(X_train, y_train, batch_size=64)
val_dataset_mw = create_dataset(X_valid, y_valid, batch_size=64)

model_multi_worker.fit(train_dataset_mw, epochs=10,
                      steps_per_epoch=len(X_train) // 64,
                      validation_data=val_dataset_mw,
                      validation_steps=len(X_valid) // 64)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.7748 - loss: 0.6374 - val_accuracy: 0.8632 - val_loss: 0.3835
Epoch 2/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8628 - loss: 0.3762 - val_accuracy: 0.8718 - val_loss: 0.3545
Epoch 3/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8800 - loss: 0.3271 - val_accuracy: 0.8852 - val_loss: 0.3131
Epoch 4/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8863 - loss: 0.3054 - val_accuracy: 0.8866 - val_loss: 0.3149
Epoch 5/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.8941 - loss: 0.2854 - val_accuracy: 0.8882 - val_loss: 0.3178
Epoch 6/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8986 - loss: 0.2661 - val_accuracy: 0.8870 - val_loss: 0.3095
Epoch 7/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9056 - loss: 0.2555 - val_accuracy: 0.8910 - val_loss: 0.3094
Epoch 8/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9085 - loss: 0.2425 - val_ac

## 3. TensorFlow Serving

### Exporting the Model (SavedModel format)

In [107]:
# Create a simple model to export
model_export = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_export.compile(loss="sparse_categorical_crossentropy",
                     optimizer="adam",
                     metrics=["accuracy"])
# Train briefly
model_export.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))

# Define the export path
# A good practice is to create a versioned directory
model_version = "001" # Or a timestamp
export_path = os.path.join("my_fashion_mnist_model_for_serving", model_version)

# Remove the directory if it already exists (for clean export)
if os.path.isdir(export_path):
    shutil.rmtree(export_path)

tf.saved_model.save(model_export, export_path)

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.7844 - loss: 0.5996 - val_accuracy: 0.8602 - val_loss: 0.3872


### Testing the SavedModel (Optional)

In [109]:
# To load and test the exported model
loaded_model = tf.saved_model.load(export_path)
print(list(loaded_model.signatures.keys())) # Should show 'serving_default'
infer = loaded_model.signatures["serving_default"]
dummy_input = tf.constant(X_test[:1].reshape(1, 28, 28), dtype=tf.float32)
# Pass the dummy_input tensor directly to the infer function
predictions = infer(dummy_input)
print(predictions)

['serving_default']
{'output_0': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[2.6912394e-05, 2.8693483e-05, 5.7107441e-06, 6.0593011e-06,
        2.3995747e-06, 3.2625917e-02, 1.2455772e-05, 4.8775226e-02,
        2.7604532e-04, 9.1824061e-01]], dtype=float32)>}


### Running TensorFlow Serving (Conceptual, requires Docker)

In [110]:
# 1. Start a Docker container for TensorFlow Serving:
#    docker pull tensorflow/serving
#    docker run -p 8500:8500 -p 8501:8501 --mount type=bind,source="$(pwd)/my_fashion_mnist_model_for_serving",target=/models/fashion_mnist -e MODEL_NAME=fashion_mnist -t tensorflow/serving &

# 2. To test with curl (HTTP REST API):
#    curl -d '{"instances": [[[0.0, ..., 0.0], ...]]]}' -X POST http://localhost:8501/v1/models/fashion_mnist:predict

# 3. Python client example (using requests library)
# import requests
# import json

# def make_prediction_request(model_name, data):
#     # Convert input data to a list of lists for JSON
#     data_list = data.tolist() # Assuming data is a NumPy array
#     request_body = {"instances": data_list}
#     headers = {"content-type": "application/json"}
#     json_response = requests.post(
#         f"http://localhost:8501/v1/models/{model_name}:predict",
#         data=json.dumps(request_body),
#         headers=headers
#     )
#     response = json.loads(json_response.text)
#     return response

# # Use a sample from X_test (reshape to 1, 28, 28)
# sample_image = X_test[0].reshape(1, 28, 28)
# predictions = make_prediction_request("fashion_mnist", sample_image)
# print(predictions)

## 4. TensorFlow Lite

### Converting a Keras Model to TensorFlow Lite (.tflite)

In [111]:
# Build a simple Keras model (already done above as model_export)
model_lite = model_export

# Create a TFLiteConverter
converter = tf.lite.TFLiteConverter.from_keras_model(model_export)

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_path = "my_fashion_mnist_model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmplhuuynhx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_310')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140366648548368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648551824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648552784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648553744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648552976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648554512: TensorSpec(shape=(), dtype=tf.resource, name=None)


### Running a TensorFlow Lite Model (on CPU, for testing)

In [118]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)

# Allocate tensors (necessary step)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data (e.g., from X_test)
# Input type might need to be cast if quantize was used (e.g., to int8)
# Reshape the input data to match the expected 3D input shape (batch_size, height, width)
# This matches the input_shape=[28, 28] in the Flatten layer definition
input_data = tf.constant(X_test[0].reshape(1, 28, 28), dtype=tf.float32)

# Set the tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Invoke the interpreter (run inference)
interpreter.invoke()

# Get the output tensor
tflite_predictions = interpreter.get_tensor(output_details[0]['index'])
# print(tflite_predictions)

### Quantization (Optimization for TFLite)

In [123]:
# Convert with default optimizations (quantization for weights)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quantized = converter.convert()

# Save the quantized model
with open("my_fashion_mnist_model_quantized.tflite", "wb") as f:
    f.write(tflite_model_quantized)

Saved artifact at '/tmp/tmpsfiph0dq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_310')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140366648548368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648551824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648552784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648553744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648552976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648554512: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [124]:
# Full integer quantization (requires representative dataset)
def representative_data_gen():
    # Iterate through the batched dataset
    for input_value in tf.data.Dataset.from_tensor_slices(X_train_full).batch(1).take(100):
        # Yield the input value directly.
        # The batch(1) ensures the shape is (1, 28, 28)
        # tf.cast to tf.float32 is already done in the previous cell for model_export.
        # However, it's good practice to ensure the data type is float32 for conversion.
        yield [tf.cast(input_value, tf.float32)]

# The rest of the conversion code remains the same
converter = tf.lite.TFLiteConverter.from_keras_model(model_export) # Re-initialize the converter if needed for fresh settings
converter.optimizations = [tf.lite.Optimize.DEFAULT] # Apply default optimizations including quantization
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8

tflite_model_full_int = converter.convert()
with open("my_fashion_mnist_model_full_int.tflite", "wb") as f:
    f.write(tflite_model_full_int)

Saved artifact at '/tmp/tmp_yr0zhr8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor_310')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140366648548368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648551824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648552784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648553744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648552976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140366648554512: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


## 5. TensorFlow.js (Conceptual, requires Node.js and browser)

In [125]:
# 1. Install tensorflowjs converter:
#    pip install tensorflowjs

# 2. Convert a SavedModel or Keras H5 model:
#    tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model \
#                           ./my_fashion_mnist_model_for_serving/001 ./tfjs_model/fashion_mnist_graph_model

#    Or for Keras H5:
#    tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model \
#                           ./my_keras_model.h5 ./tfjs_model/fashion_mnist_layers_model

# 3. Serve in a web browser (requires simple HTML/JS setup):
#    <script src="[https://cdn.jsdelivr.net/npm/@tensorflow/tfjs](https://cdn.jsdelivr.net/npm/@tensorflow/tfjs)"></script>
#    <script>
#        async function loadAndPredict() {
#            const model = await tf.loadLayersModel('tfjs_model/fashion_mnist_layers_model/model.json');
#            // Prepare input tensor (e.g., from an image)
#            const inputTensor = tf.tensor2d([[...your_28x28_pixel_data...]]);
#            const predictions = model.predict(inputTensor);
#            predictions.array().then(arr => console.log(arr));
#        }
#        loadAndPredict();
#    </script>